# Getting Started

In [2]:
# imports and setup
import os

import tasty.templates as tt
import tasty.constants as tc
import tasty.graphs as tg

## Initializations

We will start by loading in an ontology.  This returns an RDF graph, with typical namespaces loaded depending on which ontology (Brick or Haystack) we decided to load, but we don't need to worry about that too much.

In [3]:
haystack_ont = tg.load_ontology('Haystack', '3.9.9')
brick_ont = tg.load_ontology('Brick', '1.1')
print(type(haystack_ont))
print(type(brick_ont))

<class 'rdflib.graph.Graph'>
<class 'rdflib.graph.Graph'>


In [5]:
list(haystack_ont.namespaces())

[('xml', rdflib.term.URIRef('http://www.w3.org/XML/1998/namespace')),
 ('rdf', rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#')),
 ('rdfs', rdflib.term.URIRef('http://www.w3.org/2000/01/rdf-schema#')),
 ('xsd', rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#')),
 ('owl', rdflib.term.URIRef('http://www.w3.org/2002/07/owl#')),
 ('skos', rdflib.term.URIRef('http://www.w3.org/2004/02/skos/core#')),
 ('sh', rdflib.term.URIRef('http://www.w3.org/ns/shacl#')),
 ('ph', rdflib.term.URIRef('https://project-haystack.org/def/ph/3.9.9#')),
 ('phIct',
  rdflib.term.URIRef('https://project-haystack.org/def/phIct/3.9.9#')),
 ('phScience',
  rdflib.term.URIRef('https://project-haystack.org/def/phScience/3.9.9#')),
 ('phIoT',
  rdflib.term.URIRef('https://project-haystack.org/def/phIoT/3.9.9#'))]

In [6]:
list(brick_ont.namespaces())

[('xml', rdflib.term.URIRef('http://www.w3.org/XML/1998/namespace')),
 ('rdf', rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#')),
 ('rdfs', rdflib.term.URIRef('http://www.w3.org/2000/01/rdf-schema#')),
 ('xsd', rdflib.term.URIRef('http://www.w3.org/2001/XMLSchema#')),
 ('owl', rdflib.term.URIRef('http://www.w3.org/2002/07/owl#')),
 ('skos', rdflib.term.URIRef('http://www.w3.org/2004/02/skos/core#')),
 ('sh', rdflib.term.URIRef('http://www.w3.org/ns/shacl#')),
 ('brick', rdflib.term.URIRef('https://brickschema.org/schema/1.1/Brick#')),
 ('tag', rdflib.term.URIRef('https://brickschema.org/schema/1.1/BrickTag#')),
 ('bsh', rdflib.term.URIRef('https://brickschema.org/schema/1.1/BrickShape#')),
 ('dcterms', rdflib.term.URIRef('http://purl.org/dc/terms#')),
 ('sdo', rdflib.term.URIRef('http://schema.org#')),
 ('sosa', rdflib.term.URIRef('http://www.w3.org/ns/sosa#'))]

## Namespacing Terms
We will explore some core capabilities of templates by getting some typing information from a Haystack point type.  The first thing to do is to find the correct namespaces for the terms.

_Note: The way we use Namespaced terms is a Tuple data structure as: (Namespace, term)_

__Function__: `tt.get_namespaced_terms`

__Purpose__: resolve terms to namespaces in the ontology

__Description__: The function can take a simple string, with terms delimited by '-' (likely for Haystack). A Brick point type likely will just be expressed as a single class. It can also take a dictionary.  This is used when we are trying to express Datatype Properties (properties). A datatype property is a property of an entity that 'has a value'.  At this point in time, this is only really possible in Haystack (Brick I think is still in process). This has taken inspiration from the Hayson specification. When expressing fields:
- The key must be a term in the ontology
- `_kind` is a special key.  the value will be resolved to a namespaced term (i.e. number -> ph:number)
- `val` is a special key.  it expresses the desired value of the property. A NoneType (`None`) value is used to express that this field is desired, but we don't care what the value is.

In [3]:
# Recommend not putting 'properties' in the point type string (curVal), but we'll get back to this later.
point_type_string = 'cur-writable-motor-run-curVal-sensor-point'
haystack_namespaced_terms = tt.get_namespaced_terms(haystack_ont, point_type_string)

brick_type = 'Damper_Position_Command'
brick_namespaced_terms = tt.get_namespaced_terms(brick_ont, brick_type)

# expressing datatype properties via a dict
properties = {
    'curVal': {
        '_kind': 'number',
        'val': None
    },
    # another form
    'unit': 'cfm'
}

namespaced_properties = tt.get_namespaced_terms(haystack_ont, properties)

# types have two values in each tuple: (Namespace, term)
print(brick_namespaced_terms)

# properties have three values in each tuple: (Namespace, term, dict)
print(namespaced_properties)

{(Namespace('https://brickschema.org/schema/1.1/Brick#'), 'Damper_Position_Command')}
{(Namespace('https://project-haystack.org/def/ph/3.9.9#'), 'unit', <frozendict {'val': 'cfm'}>), (Namespace('https://project-haystack.org/def/phIoT/3.9.9#'), 'curVal', <frozendict {'_kind': (Namespace('https://project-haystack.org/def/ph/3.9.9#'), 'number'), 'val': None}>)}


## Determining Classes

The point of using Semantic Web technologies is to encode the 'type' or 'class' of something into the ontology. From the perspective of Brick, we have likely already done this by getting a namespaced term.  With Haystack, typing is a bit more nuanced, and refers more to the expected functionality (i.e. describing what it _implements_ as well as what it is).  For example, folks used to using sets of markers for expressing the point type above (`cur-air-writable-motor-curVal-sensor-point`) would know they expect to be able to:
- get the current value of this point - it is a `phIoT:cur-point`
- write to this point - it is a `phIoT:writable-point`
- it somehow also represents a type of thing that is a motor - `phIoT:motor`

We solve this with a simple function call:

__Function__: `tt.hget_entity_classes`

__Purpose__: determine (from a set of namespaced terms) the different classes implemented.

In [4]:
structured_terms = tt.hget_entity_classes(haystack_ont, haystack_namespaced_terms)

[tasty.templates.hget_entity_classes] Permutation time: 5.04 seconds


A dictionary is returned with three keys, describing the information determined:
- 'classes'
- 'markers'
- 'properties'

Print them out to see the exact information contained in each

In [5]:
print(structured_terms['classes'])

{(Namespace('https://project-haystack.org/def/phIoT/3.9.9#'), 'cur-point'), (Namespace('https://project-haystack.org/def/phIoT/3.9.9#'), 'writable-point'), (Namespace('https://project-haystack.org/def/phIoT/3.9.9#'), 'motor')}


In [6]:
print(structured_terms['markers'])

{(Namespace('https://project-haystack.org/def/phIoT/3.9.9#'), 'sensor'), (Namespace('https://project-haystack.org/def/phIoT/3.9.9#'), 'run')}


In [7]:
# Here is where we pull out namespaced terms that are not things, but which are expected to contain values.
# They are automatically assigned a 'val': None
print(structured_terms['properties'])

{(Namespace('https://project-haystack.org/def/phIoT/3.9.9#'), 'curVal', <frozendict {'val': None}>)}
